In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch 
import torch.nn as nn
import time
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader

In [ ]:
# load brain_extraction data
X_Guys = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/X_Guys.npy')
y_Guys = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/y_Guys.npy')
ids_Guys = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/ids_Guys.npy')
X_HH = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/X_HH.npy')
y_HH = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/y_HH.npy')
ids_HH = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/ids_HH.npy')
X_IOP = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/X_IOP.npy')
y_IOP = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/y_IOP.npy')
ids_IOP = np.load('/content/drive/MyDrive/dhl_exam/data/brain_extraction/ids_IOP.npy')

In [ ]:
#center data
def centring(X):
    X=np.asarray(X)
    epsilon = 1e-7 
    mean = np.mean(X, axis=0, keepdims=True)
    std = np.std(X, axis=0, keepdims=True)
    centered_array = (X - mean) / (std+epsilon)
    return centered_array

X_Guys_centered=centring(X_Guys)
X_HH_centered=centring(X_HH)
X_IOP_centered=centring(X_IOP)

In [ ]:
# producing required train / val / test split
print("initial shapes")
print(X_Guys.shape)
print(y_Guys.shape)
print(ids_Guys.shape)
print(X_HH.shape)
print(y_HH.shape)
print(ids_HH.shape)
print(X_IOP.shape)
print(y_IOP.shape)
print(ids_IOP.shape)

combined_Guys_HH_X = np.concatenate([X_Guys_centered, X_HH_centered], axis=0)
combined_Guys_HH_y = np.concatenate([y_Guys, y_HH], axis=0)

X_train = torch.Tensor(combined_Guys_HH_X[0:int(len(combined_Guys_HH_X)*0.85)])
y_train = torch.Tensor(combined_Guys_HH_y[0:int(len(combined_Guys_HH_y)*0.85)])

# val data 15%
X_val = torch.Tensor(combined_Guys_HH_X[int(len(combined_Guys_HH_X)*0.85):int(len(combined_Guys_HH_X))])
y_val = torch.Tensor(combined_Guys_HH_y[int(len(combined_Guys_HH_y)*0.85):int(len(combined_Guys_HH_y))])
#test data from IOP data
X_test =  torch.Tensor(X_IOP_centered[0:int(len(X_IOP_centered))])
y_test = torch.Tensor(y_IOP[0:int(len(y_IOP))])

print("Check after split")
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

initial shapes
(317, 40, 128, 128)
(317, 40, 128, 128)
(317,)
(176, 40, 128, 128)
(176, 40, 128, 128)
(176,)
(71, 40, 128, 128)
(71, 40, 128, 128)
(71,)
Check after split
torch.Size([419, 40, 128, 128])
torch.Size([419, 40, 128, 128])
torch.Size([74, 40, 128, 128])
torch.Size([74, 40, 128, 128])
torch.Size([71, 40, 128, 128])
torch.Size([71, 40, 128, 128])


In [ ]:
#sample slices - just pick 1 slice from 40 so for training always 
X_train=X_train[:,10,:,:]
X_val=X_val[:,20,:,:]
X_test=X_test[:,30,:,:]
y_train=y_train[:,10,:,:]
y_val=y_val[:,20,:,:]
y_test=y_test[:,30,:,:]

#introduce channel
X_train=torch.reshape(X_train,(len(X_train),1,128,128))
X_val=torch.reshape(X_val,(len(X_val),1,128,128))
X_test=torch.reshape(X_test,(len(X_test),1,128,128))
y_train=torch.reshape(y_train,(len(y_train),1,128,128))
y_val=torch.reshape(y_val,(len(y_val),1,128,128))
y_test=torch.reshape(y_test,(len(y_test),1,128,128))


print("Check shapes after slicing into 2D")
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

Check shapes after slicing into 2D
torch.Size([419, 1, 128, 128])
torch.Size([419, 1, 128, 128])
torch.Size([74, 1, 128, 128])
torch.Size([74, 1, 128, 128])
torch.Size([71, 1, 128, 128])
torch.Size([71, 1, 128, 128])


In [ ]:
def to_one_hot(y, num_classes):
    y = np.array(y, dtype='int')
    one_hot = np.eye(num_classes)[y.flatten()]
    return one_hot.reshape(y.shape[0], num_classes,y.shape[2],y.shape[3]).astype(float)

y_train = to_one_hot(y_train, 2)
y_val= to_one_hot(y_val, 2)
y_test = to_one_hot(y_test, 2)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(419, 2, 128, 128)
(74, 2, 128, 128)
(71, 2, 128, 128)


In [ ]:
X_train=np.asarray(X_train)
X_val=np.asarray(X_val)
X_test=np.asarray(X_test)

class numpy_dataset(Dataset): 
    def __init__(self, data, target): 
        self.data =  torch.from_numpy(data)
        self.target = torch.from_numpy(target)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y

    def __len__(self):
        return len(self.data)
    
train_dataset = numpy_dataset(X_train, y_train)
val_dataset = numpy_dataset(X_val, y_val)
test_dataset = numpy_dataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, drop_last=True)


In [ ]:
# adapted from Kaggle
def dice_coef_metric(pred, label):
    intersection = 2.0 * (pred * label).sum()
    union = pred.sum() + label.sum()
    if pred.sum() == 0 and label.sum() == 0:
        return 1.
    return intersection / union

def dice_coef_loss(pred, label):
    smooth = 1.0
    intersection = 2.0 * (pred * label).sum() + smooth
    union = pred.sum() + label.sum() + smooth
    return 1 - (intersection / union)

def bce_dice_loss(pred, label):
    dice_loss = dice_coef_loss(pred, label)
    bce_loss = nn.BCELoss()(pred, label)
    return dice_loss + bce_loss


def tversky(y_pred,y_true, smooth=1, alpha=0.7):
    y_true_pos = y_true.view(-1)
    y_pred_pos = y_pred.view(-1)
    true_pos = torch.sum(y_true_pos * y_pred_pos)
    false_neg = torch.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = torch.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)


def tversky_loss(y_pred,y_true):
    return 1 - tversky(y_true, y_pred)


def focal_tversky_loss(y_pred,y_true, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return torch.pow((1 - tv), gamma)


def lovasz_softmax_flat(logits, labels):
    probas = F.softmax(logits, dim=1)
    labels = labels.float()

    if probas.numel() == 0:
        return logits * 0.0

    signs = 2 * labels - 1
    errors = (1 - probas * signs)
    errors_sorted, perm = torch.sort(errors.reshape(-1), dim=0, descending=True)
    perm = perm.cpu().numpy()
    gt_sorted = labels.reshape(-1)[perm]
    grad = lovasz_grad(gt_sorted)

    loss = torch.dot(F.elu(errors_sorted) + 1, Variable(grad, requires_grad=False))
    return loss

def lovasz_grad(gt_sorted):

    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.cumsum(0)
    union = gts + (1 - gt_sorted).cumsum(0)
    jaccard = 1.0 - intersection / union
    if p > 1:  # cover cases where p == 0
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard





In [ ]:
lf_train_loss_history={}
lf_train_dice_history={}
lf_val_loss_history={}
lf_val_dice_history={}
dict_pred_dice={}
dict_stopping_epoch={}
dict_time_taken={}

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lfs=[dice_coef_loss,bce_dice_loss,tversky_loss,focal_tversky_loss,lovasz_softmax_flat]

for lf in lfs:
  start_time = time.time()
  model = UNet(n_channels=1, n_classes=2, bilinear=True).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
  train_loss_history, train_dice_history, val_loss_history, val_dice_history,se = train_model_early_stopping(train_dataloader, val_dataloader, lf, optimizer, scheduler, num_epochs)
  end_time = time.time()
  lf_train_loss_history[str(lf)]=train_loss_history
  lf_train_dice_history[str(lf)]=train_dice_history
  lf_val_loss_history[str(lf)]=val_loss_history
  lf_val_dice_history[str(lf)]=val_dice_history
  dict_pred_dice[str(lf)]=prediction_dice(model, test_dataloader)
  dict_stopping_epoch[str(lf)]=se
  dict_time_taken[str(lf)]=end_time - start_time

In [ ]:
def plot_dice_history(model_name, train_dice_history, val_dice_history, num_epochs):
    
    x = np.arange(num_epochs)
    fig = plt.figure(figsize=(10, 6))
    plt.plot(x, train_dice_history, label='Training DICE Score', lw=3)
    plt.plot(x, val_dice_history, label='Validation DICE Score', lw=3)

    plt.title(f"{model_name}", fontsize=20)
    plt.legend(fontsize=12)
    plt.xlabel("Epoch", fontsize=15)
    plt.ylabel("DICE score", fontsize=15)

    path='/content/drive/MyDrive/dhl_exam/figures/'+str(model_name)+"-dice.png"
    plt.savefig(path)
    plt.show()
    

def plot_loss_history(model_name, train_loss_history, val_loss_history, num_epochs):
    
    x = np.arange(num_epochs)
    fig = plt.figure(figsize=(10, 6))
    plt.plot(x, train_loss_history, label='Training Loss', lw=3)
    plt.plot(x, val_loss_history, label='Validation Loss', lw=3)

    plt.title(f"{model_name}", fontsize=20)
    plt.legend(fontsize=12)
    plt.xlabel("Epoch", fontsize=15)
    plt.ylabel("Loss", fontsize=15)
    path='/content/drive/MyDrive/dhl_exam/figures/'+str(model_name)+"-loss.png"
    plt.savefig(path)
    plt.show()
    

def prediction_dice(net, test_dataloader):
    test_dice=0

    with torch.no_grad():  # So no gradients accumulate
        for batch_idx, (data, target) in enumerate(test_dataloader):
          data = data.to(device).float()
          target = target.to(device).float()
           
          pred = net(data)
          out_cut = np.copy(pred.data.cpu().numpy())
          out_cut[np.nonzero(out_cut < 0.5)] = 0.0
          out_cut[np.nonzero(out_cut >= 0.5)] = 1.0
          dice = dice_coef_metric(out_cut, target.data.cpu().numpy())
          test_dice += dice
        mean_dice = test_dice / len(test_dataloader)
        return mean_dice

def predict(net, test_dataloader):
    test_dice=0

    with torch.no_grad():  # So no gradients accumulate
        for batch_idx, (data, target) in enumerate(test_dataloader):
          data = data.to(device).float()
          target = target.to(device).float()
           
          pred = net(data)
    return data.data.cpu().numpy(),target.data.cpu().numpy(),pred.data.cpu().numpy()


In [ ]:
def train_loop(model, loader, loss_func,optimizer):
    model.train()
    train_losses = []
    train_dices = []
    
#     
    for i, (image, mask) in enumerate(loader):
        image = image.to(device).float()
        mask = mask.to(device).float()
        outputs = model(image)
        out_cut = np.copy(outputs.data.cpu().numpy())
        out_cut[np.nonzero(out_cut < 0.5)] = 0.0
        out_cut[np.nonzero(out_cut >= 0.5)] = 1.0            

        dice = dice_coef_metric(out_cut, mask.data.cpu().numpy())
        loss = loss_func(outputs, mask)
        train_losses.append(loss.item())
        train_dices.append(dice)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    return train_dices, train_losses

def eval_loop(model, loader, loss_func, scheduler,training=True):
    model.eval()
    val_loss = 0
    val_dice = 0
    with torch.no_grad():
        for step, (image, mask) in enumerate(loader):
            image = image.to(device).float()
            mask = mask.to(device).float()
    
            outputs = model(image)
            loss = loss_func(outputs, mask)
            
            out_cut = np.copy(outputs.data.cpu().numpy())
            out_cut[np.nonzero(out_cut < 0.5)] = 0.0
            out_cut[np.nonzero(out_cut >= 0.5)] = 1.0
            dice = dice_coef_metric(out_cut, mask.data.cpu().numpy())
            
            val_loss += loss
            val_dice += dice
        
        val_mean_dice = val_dice / len(loader)
        val_mean_loss = val_loss / step
        
        if training:
            scheduler.step(val_mean_dice)
        
    return val_mean_dice, val_mean_loss

def train_model(train_loader, val_loader, loss_func, optimizer, scheduler, num_epochs):
    train_loss_history = []
    train_dice_history = []
    val_loss_history = []
    val_dice_history = []
    
    for epoch in range(num_epochs):
        train_dices, train_losses = train_loop(model, train_loader, loss_func,optimizer)
        train_mean_dice = np.array(train_dices).mean()
        train_mean_loss = np.array(train_losses).mean()
        val_mean_dice, val_mean_loss = eval_loop(model, val_loader, loss_func,scheduler)
        
        train_loss_history.append(np.array(train_losses).mean())
        train_dice_history.append(np.array(train_dices).mean())
        val_loss_history.append(val_mean_loss.cpu().numpy())
        val_dice_history.append(val_mean_dice)
        
        print('Epoch: {}/{} |  Train Loss: {:.3f}, Val Loss: {:.3f}, Train DICE: {:.3f}, Val DICE: {:.3f}'.format(epoch+1, num_epochs,
                                                                                                                 train_mean_loss,
                                                                                                                 val_mean_loss,
                                                                                                                 train_mean_dice,
                                                                                                                 val_mean_dice))
        

    return train_loss_history, train_dice_history, val_loss_history, val_dice_history

def train_model_early_stopping(train_loader, val_loader, loss_func, optimizer, scheduler, num_epochs, patience=5):
    train_loss_history = []
    train_dice_history = []
    val_loss_history = []
    val_dice_history = []

    best_val_dice = 0
    consecutive_no_improvement = 0
    
    for epoch in range(num_epochs):
        train_dices, train_losses = train_loop(model, train_loader, loss_func, optimizer)
        train_mean_dice = np.array(train_dices).mean()
        train_mean_loss = np.array(train_losses).mean()
        val_mean_dice, val_mean_loss = eval_loop(model, val_loader, loss_func, scheduler)
        
        train_loss_history.append(train_mean_loss)
        train_dice_history.append(train_mean_dice)
        val_loss_history.append(val_mean_loss.cpu().numpy())
        val_dice_history.append(val_mean_dice)
        
        print('Epoch: {}/{} |  Train Loss: {:.3f}, Val Loss: {:.3f}, Train DICE: {:.3f}, Val DICE: {:.3f}'.format(epoch+1, num_epochs,
                                                                                                                 train_mean_loss,
                                                                                                                 val_mean_loss,
                                                                                                                 train_mean_dice,
                                                                                                                 val_mean_dice))

        if val_mean_dice > best_val_dice:
            best_val_dice = val_mean_dice
            consecutive_no_improvement = 0
            print('Best validation dice coefficient improved to {:.3f}'.format(best_val_dice))
        else:
            consecutive_no_improvement += 1
            print('No improvement in validation dice coefficient for {} consecutive epochs'.format(consecutive_no_improvement))
            if consecutive_no_improvement >= patience:
                print('Early stopping triggered after {} epochs'.format(epoch+1))
                break

    return train_loss_history, train_dice_history, val_loss_history, val_dice_history,epoch+1


In [ ]:
class FCN_flexible(nn.Module):
    def __init__(self, input_shape=(1, 128, 128), num_classes=2, dropout_prob=0.5, num_layers=2):
        super(FCN_flexible, self).__init__()

       
        encoder_layers = []
        in_channels = input_shape[0]
        for i in range(num_layers):
            out_channels = 64 * (2 ** i)
            encoder_layers.extend([
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_prob),
                nn.MaxPool2d(kernel_size=2, stride=2)
            ])
            in_channels = out_channels

        self.encoder = nn.Sequential(*encoder_layers)

        
        self.middle = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * 2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_prob),
            nn.Conv2d(in_channels * 2, in_channels, kernel_size=1)
        )

       
        decoder_layers = []
        for i in range(num_layers - 1, -1, -1):
            out_channels = 64 * (2 ** i)
            decoder_layers.extend([
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_prob)
            ])
            in_channels = out_channels

        decoder_layers.extend([
            nn.Conv2d(out_channels, num_classes, kernel_size=1),
            nn.Sigmoid()
        ])

        self.decoder = nn.Sequential(*decoder_layers)

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x



In [ ]:

def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.pool = nn.MaxPool2d(2)
        self.conv = conv_block(in_channels, out_channels)

    def forward(self, x):
        return self.conv(self.pool(x))

class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = conv_block(in_channels, out_channels)

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat((skip, x), dim=1)
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.init_conv = conv_block(in_channels, 64)
        self.encoders = nn.ModuleList([
            Encoder(64, 128),
            Encoder(128, 256),
            Encoder(256, 512),
            Encoder(512, 1024)
        ])
        self.decoders = nn.ModuleList([
            Decoder(1024, 512),
            Decoder(512, 256),
            Decoder(256, 128),
            Decoder(128, 64)
        ])
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.init_conv(x)
        skips = [x1]
        for encoder in self.encoders:
            skips.append(encoder(skips[-1]))

        x = skips.pop()
        for decoder in self.decoders:
            x = decoder(x, skips.pop())

        return self.final_conv(x)


In [ ]:
import itertools

alphas = [0.3, 0.5, 0.7]
betas = [0.3, 0.5, 0.7]
smooths = [1.0, 1.5, 2.0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# grid search experiment
results = []
for alpha, beta, smooth in itertools.product(alphas, betas, smooths):
    print(f'Training TverskyLoss with alpha={alpha}, beta={beta}, smooth={smooth}...')
    loss_func = TverskyLoss(alpha=alpha, beta=beta, smooth=smooth)
    model = UNet(n_channels=1, n_classes=2, bilinear=True).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
    train_loss_history, train_dice_history, val_loss_history, val_dice_history, epochs = train_model_early_stopping(train_dataloader, val_dataloader, loss_func, optimizer, scheduler, num_epochs=50)
    results.append((alpha, beta, smooth, epochs, max(val_dice_history)))


print('\nGrid Search Results:')
for alpha, beta, smooth, epochs, val_dice in results:
    print(f'alpha={alpha}, beta={beta}, smooth={smooth}, epochs={epochs}, val_dice={val_dice:.3f}')

In [ ]:
num_epochs=50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(n_channels=1, n_classes=2, bilinear=False).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
train_loss_history, train_dice_history, val_loss_history, val_dice_history, e= train_model_early_stopping(train_dataloader, val_dataloader, tversky_loss, optimizer, scheduler, num_epochs)

Epoch: 1/50 |  Train Loss: 0.911, Val Loss: 1.207, Train DICE: 0.719, Val DICE: 0.872
Best validation dice coefficient improved to 0.872
Epoch: 2/50 |  Train Loss: 0.863, Val Loss: 1.140, Train DICE: 0.915, Val DICE: 0.914
Best validation dice coefficient improved to 0.914
Epoch: 3/50 |  Train Loss: 0.841, Val Loss: 1.129, Train DICE: 0.946, Val DICE: 0.928
Best validation dice coefficient improved to 0.928
Epoch: 4/50 |  Train Loss: 0.828, Val Loss: 1.109, Train DICE: 0.958, Val DICE: 0.943
Best validation dice coefficient improved to 0.943
Epoch: 5/50 |  Train Loss: 0.818, Val Loss: 1.100, Train DICE: 0.964, Val DICE: 0.946
Best validation dice coefficient improved to 0.946
Epoch: 6/50 |  Train Loss: 0.810, Val Loss: 1.094, Train DICE: 0.968, Val DICE: 0.943
No improvement in validation dice coefficient for 1 consecutive epochs
Epoch: 7/50 |  Train Loss: 0.803, Val Loss: 1.082, Train DICE: 0.971, Val DICE: 0.950
Best validation dice coefficient improved to 0.950
Epoch: 8/50 |  Train 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs=50
dict_lf_to_parameters={}
lf_train_loss_history={}
lf_train_dice_history={}
lf_val_loss_history={}
lf_val_dice_history={}
dict_pred_dice={}
dict_stopping_epoch={}
dict_time_taken={}

# loss functions to compare
loss_funcs = [dice_coef_loss,bce_dice_loss,TverskyLoss(),FocalLoss(),lovasz_softmax_flat]
for lf in loss_funcs:
  start_time = time.time()
  model = FCN_flexible(input_shape=(1, 128, 128), num_classes=2, dropout_prob=0.5, num_layers=5).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
  train_loss_history, train_dice_history, val_loss_history, val_dice_history, e= train_model_early_stopping(train_dataloader, val_dataloader, lf, optimizer, scheduler, num_epochs)
  dict_pred_dice[lf]=prediction_dice(model, test_dataloader)
  end_time = time.time()
  lf_train_loss_history[lf]=train_loss_history
  lf_train_dice_history[lf]=train_dice_history
  lf_val_loss_history[lf]=val_loss_history
  lf_val_dice_history[lf]=val_dice_history
  dict_stopping_epoch[lf]=e
  dict_time_taken[lf]=end_time - start_time
  

Epoch: 1/50 |  Train Loss: 0.325, Val Loss: 0.385, Train DICE: 0.688, Val DICE: 0.711
Best validation dice coefficient improved to 0.711
Epoch: 2/50 |  Train Loss: 0.298, Val Loss: 0.385, Train DICE: 0.702, Val DICE: 0.711
No improvement in validation dice coefficient for 1 consecutive epochs
Epoch: 3/50 |  Train Loss: 0.298, Val Loss: 0.385, Train DICE: 0.702, Val DICE: 0.711
No improvement in validation dice coefficient for 2 consecutive epochs
Epoch: 4/50 |  Train Loss: 0.299, Val Loss: 0.388, Train DICE: 0.701, Val DICE: 0.709
No improvement in validation dice coefficient for 3 consecutive epochs
Epoch: 5/50 |  Train Loss: 0.300, Val Loss: 0.388, Train DICE: 0.700, Val DICE: 0.709
No improvement in validation dice coefficient for 4 consecutive epochs
Epoch: 6/50 |  Train Loss: 0.300, Val Loss: 0.388, Train DICE: 0.700, Val DICE: 0.709
No improvement in validation dice coefficient for 5 consecutive epochs
Early stopping triggered after 6 epochs
Epoch: 1/50 |  Train Loss: 1.627, Val 

/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1/50 |  Train Loss: 0.406, Val Loss: 0.532, Train DICE: 0.689, Val DICE: 0.711
Best validation dice coefficient improved to 0.711
Epoch: 2/50 |  Train Loss: 0.401, Val Loss: 0.532, Train DICE: 0.702, Val DICE: 0.711
No improvement in validation dice coefficient for 1 consecutive epochs
Epoch: 3/50 |  Train Loss: 0.401, Val Loss: 0.532, Train DICE: 0.702, Val DICE: 0.711
No improvement in validation dice coefficient for 2 consecutive epochs
Epoch: 4/50 |  Train Loss: 0.401, Val Loss: 0.532, Train DICE: 0.703, Val DICE: 0.711
No improvement in validation dice coefficient for 3 consecutive epochs
Epoch: 5/50 |  Train Loss: 0.401, Val Loss: 0.532, Train DICE: 0.702, Val DICE: 0.711
No improvement in validation dice coefficient for 4 consecutive epochs
Epoch: 6/50 |  Train Loss: 0.401, Val Loss: 0.532, Train DICE: 0.702, Val DICE: 0.711
No improvement in validation dice coefficient for 5 consecutive epochs
Early stopping triggered after 6 epochs
Epoch: 1/50 |  Train Loss: 0.556, Val 

In [ ]:
for lf in loss_funcs:

  plot_dice_history('DICE FCN with {}'.format(lf), lf_train_dice_history[lf], lf_val_dice_history[lf], len(lf_val_dice_history[lf]))
  plot_loss_history('LOSS FCN with {}'.format(lf), lf_train_loss_history[lf], lf_val_loss_history[lf],len(lf_val_loss_history[lf]))

In [ ]:
import itertools

# Define the range of values to search over
alphas = [0.3, 0.5, 0.7]
betas = [0.3, 0.5, 0.7]
smooths = [1.0, 1.5, 2.0]

# Perform the grid search experiment
results = []
for alpha, beta, smooth in itertools.product(alphas, betas, smooths):
    print(f'Training TverskyLoss with alpha={alpha}, beta={beta}, smooth={smooth}...')
    loss_func = TverskyLoss(alpha=0.7, beta=0.7, smooth=smooth)
    model = FCN_flexible(input_shape=(1, 128, 128), num_classes=2, dropout_prob=0.5, num_layers=4).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5, verbose=True)
    train_loss_history, train_dice_history, val_loss_history, val_dice_history, epochs = train_model_early_stopping(train_dataloader, val_dataloader, loss_func, optimizer, scheduler, num_epochs=50)

    train_loss_history, train_dice_history, val_loss_history, val_dice_history, epochs = train_model_early_stopping(train_dataloader, val_dataloader, loss_func, optimizer, scheduler, num_epochs=50)
    results.append((alpha, beta, smooth, epochs, max(val_dice_history)))

# Print the results of the grid search
print('\nGrid Search Results:')
for alpha, beta, smooth, epochs, val_dice in results:
    print(f'alpha={alpha}, beta={beta}, smooth={smooth}, epochs={epochs}, val_dice={val_dice:.3f}')